# Modelo de Sequence-to-Sequence
### Abaixo segue a implementação basica de um modelo 'Sequence-to-Sequence' (Seq2Seq) que é um tipo de modelo que utiliza conceitos de encoder e decoder para tarefas de tradução ou geração de texto. No exemplo o objetivo e treinar o modelo para traduzir uma frase do inglês para português

Mais informações sobre modelos Seq2Seq em: 
https://medium.com/luisfredgs/o-que-%C3%A9-sequence-to-sequence-em-deep-learning-9f8857a423ca

In [72]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [110]:
# dados de exemplo em inglês e português
texto_entrada = ['hello', 'how are you', 'goodbye']
texto_alvo = ['olá', 'como você está', 'adeus']

In [111]:
# adicionando tokens de início e fim para as frases de destino
texto_alvo_entrada = ['\t' + text for text in texto_alvo]
texto_alvo_saida = [text + '\n' for text in texto_alvo]

In [112]:
# realizando a tokenização do texto de entrada
tokenizer_entrada = Tokenizer(char_level=False)
tokenizer_entrada.fit_on_texts(texto_entrada) # ajusta o tokenizador 
sequencias_entrada = tokenizer_entrada.texts_to_sequences(texto_entrada) # converte os textos de entrada para inteiros
sequencias_entrada = pad_sequences(sequencias_entrada, padding='post')

# realizando a tokenização do texto de saída
tokenizer_alvo = Tokenizer(char_level=False)
tokenizer_alvo.fit_on_texts(texto_alvo_entrada + texto_alvo_saida)
sequencia_alvo_entrada = tokenizer_alvo.texts_to_sequences(texto_alvo_entrada)
sequencia_alvo_saida = tokenizer_alvo.texts_to_sequences(texto_alvo_saida)
sequencia_alvo_entrada = pad_sequences(sequencia_alvo_entrada, padding='post')
sequencia_alvo_saida = pad_sequences(sequencia_alvo_saida, padding='post')

In [113]:
# definindo os parâmetros do modelo
num_encoder_tokens = len(tokenizer_entrada.word_index) + 1
num_decoder_tokens = len(tokenizer_alvo.word_index) + 1
dimensão_lat = 50  # dimensão do espaço latente

In [114]:
# definindo o encoder
encoder_entrada = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=dimensão_lat)(encoder_entrada) # realizando o embedding 
encoder_lstm = LSTM(dimensão_lat, return_state=True)
encoder_saida, estado_h, estado_c = encoder_lstm(encoder_embedding)
encoder_estado = [estado_h, estado_c]

In [115]:
# definindo o decoder
decoder_entrada = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=num_decoder_tokens, output_dim=dimensão_lat)(decoder_entrada) # realizando o embedding 
decoder_lstm = LSTM(dimensão_lat, return_sequences=True, return_state=True)
decoder_saida, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_estado )
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_saida = decoder_dense(decoder_saida)

In [116]:
# criando o modelo Seq2Seq
modelo = Model([encoder_entrada, decoder_entrada], decoder_saida)
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [117]:
# ajustando os dados para treinamento
decoder_alvos = np.expand_dims(sequencia_alvo_saida, -1)

# realizando o treinamento do modelo
modelo.fit([sequencias_entrada, sequencia_alvo_entrada], decoder_alvos,
          batch_size=1,
          epochs=50,
          validation_split=0.2)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - loss: 1.7900 - val_loss: 1.7717
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.7799 - val_loss: 1.7650
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1.7643 - val_loss: 1.7552
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.7603 - val_loss: 1.7467
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.7499 - val_loss: 1.7371
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.7387 - val_loss: 1.7263
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 1.7095 - val_loss: 1.7126
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.7137 - val_loss: 1.6993
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.6992 - val_loss: 1.6841
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.6829 - val_loss: 1.6667
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.6647 - val_loss: 1.6469
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.6002 - val_loss: 1.6218


In [118]:
# definindo o modelo de inferência
encoder_modelo = Model(encoder_entrada, encoder_estado )

decoder_estado_entrada_h = Input(shape=(dimensão_lat,))
decoder_estado_entrada_c = Input(shape=(dimensão_lat,))
decoder_estados_entradas = [decoder_estado_entrada_h, decoder_estado_entrada_c]
decoder_embedding_inf = Embedding(input_dim=num_decoder_tokens, output_dim=dimensão_lat)(decoder_entrada)
decoder_saidas_inf, estado_h_inf, estado_c_inf = decoder_lstm(decoder_embedding_inf, initial_state=decoder_estados_entradas)
decoder_saidas_inf = decoder_dense(decoder_saidas_inf)
decoder_estados  = [estado_h_inf, estado_c_inf]

modelo_decoder = Model([decoder_entrada] + decoder_estados_entradas, [decoder_saidas_inf] + decoder_estados )

In [119]:
# definindo uma função para decodificar uma sequência
def decodificador_seq(seq_entrada):
    valores_estados = encoder_modelo.predict(seq_entrada) # obtem os estados iniciais do encoder a partir da sequência de entrada
    seq_alvo = np.zeros((1, 1))
    seq_alvo[0, 0] = tokenizer_alvo.word_index.get('\t', 0)  # início da sequência de destino
    

    # variaveis de controle para o loop de decodificação
    cond_parada = False
    sentenca_decodificada = ''
    iteraçoes_lim = 100  # limite de iterações para evitar loop infinito
    iteracoes = 0
    palavra_anterior = ''
    

    # loop de decodificação
    while not cond_parada:
        tokens_saida, h, c = modelo_decoder.predict([seq_alvo] + valores_estados) # faz a predição do próximo token com o decoder
        indice_token  = np.argmax(tokens_saida[0, -1, :])
        amostra_palavra  = tokenizer_alvo.index_word.get(indice_token , '')
        
        if amostra_palavra  == '\n' or len(sentenca_decodificada.split()) > 50 or iteracoes > iteraçoes_lim or amostra_palavra  == palavra_anterior:
            cond_parada = True
        
        sentenca_decodificada += ' ' + amostra_palavra   # adiciona a palavra decodificada à sentença final
        seq_alvo = np.zeros((1, 1))
        seq_alvo[0, 0] = indice_token 
        valores_estados = [h, c]
        iteracoes += 1
        palavra_anterior = amostra_palavra 
    
    return sentenca_decodificada.strip()

In [120]:
# testando a decodificação
sequencia_teste = pad_sequences(tokenizer_entrada.texts_to_sequences(['how are you']), maxlen=sequencias_entrada.shape[1])
sentenca_decodificada = decodificador_seq(sequencia_teste)
print('Sentença decodificada:', sentenca_decodificada)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Sentença decodificada: como você está está
